# Neighborly Functions App - Cloud Developer With MS Azure - Udacity Project II

## 1. Add configuration

In [26]:
import configparser

In [27]:
config = configparser.ConfigParser()
config.read_file(open('azure.cfg'))

RESOURCE_GROUP                          = config.get('RESOURCE','RESOURCE_GROUP')
LOCATION                                = config.get('RESOURCE','LOCATION')
COSMOSDB_ACCOUNT_NAME                   = config.get("DATABASE","COSMOSDB_ACCOUNT_NAME")
COSMOSDB_KIND                           = config.get("DATABASE","COSMOSDB_KIND")
COSMOSDB_SERVER_VERSION                 = config.get("DATABASE","COSMOSDB_SERVER_VERSION")
COSMOSDB_DEFAULT_CONSISTENCY_LEVEL      = config.get("DATABASE","COSMOSDB_DEFAULT_CONSISTENCY_LEVEL")
COSMOSDB_ENABLE_AUTOMATIC_FAILOVER      = config.get("DATABASE","COSMOSDB_ENABLE_AUTOMATIC_FAILOVER")
COSMOSDB_DATBASE_NAME                   = config.get("DATABASE","COSMOSDB_DATABASE_NAME")
STORAGE_ACCOUNT_NAME                    = config.get("STORAGE","STORAGE_ACCOUNT_NAME")
STORAGE_SKU                             = config.get("STORAGE","STORAGE_SKU")
APP_NAME                                = config.get("APP","APP_NAME")
APP_SERVICE_PLAN_NAME                   = config.get("APP","APP_SERVICE_PLAN_NAME")
APP_SKU                                 = config.get("APP","APP_SKU")
APP_RUNTIME                             = config.get("APP","APP_RUNTIME")
FUNCTION_NAME                           = config.get("FUNCTION","FUNCTION_NAME")
FUNCTION_VERSION                        = config.get("FUNCTION","FUNCTION_VERSION")
FUNCTION_OS_TYPE                        = config.get("FUNCTION","FUNCTION_OS_TYPE")
FUNCTION_RUNTIME                        = config.get("FUNCTION","FUNCTION_RUNTIME")

In [15]:
import os
os.environ["RESOURCE_GROUP"]                        = RESOURCE_GROUP
os.environ["LOCATION"]                              = LOCATION
os.environ["COSMOSDB_ACCOUNT_NAME"]                 = COSMOSDB_ACCOUNT_NAME
os.environ["COSMOSDB_KIND"]                         = COSMOSDB_KIND
os.environ["COSMOSDB_SERVER_VERSION"]               = COSMOSDB_SERVER_VERSION
os.environ["COSMOSDB_DEFAULT_CONSISTENCY_LEVEL"]    = COSMOSDB_DEFAULT_CONSISTENCY_LEVEL
os.environ["COSMOSDB_ENABLE_AUTOMATIC_FAILOVER"]    = COSMOSDB_ENABLE_AUTOMATIC_FAILOVER
os.environ["COSMOSDB_DATBASE_NAME"]                 = COSMOSDB_DATBASE_NAME
os.environ["STORAGE_ACCOUNT_NAME"]                  = STORAGE_ACCOUNT_NAME
os.environ["STORAGE_SKU"]                           = STORAGE_SKU
os.environ["APP_NAME"]                              = APP_NAME
os.environ["APP_SERVICE_PLAN_NAME"]                 = APP_SERVICE_PLAN_NAME
os.environ["APP_SKU"]                               = APP_SKU
os.environ["APP_RUNTIME"]                           = APP_RUNTIME
os.environ["FUNCTION_NAME"]                         = FUNCTION_NAME
os.environ["FUNCTION_VERSION"]                      = FUNCTION_VERSION
os.environ["FUNCTION_OS_TYPE"]                      = FUNCTION_OS_TYPE
os.environ["FUNCTION_RUNTIME"]                      = FUNCTION_RUNTIME

## 2. Create Resources

### 2.1. Login Azure CLI

In [2]:
! az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "f958e84a-92b8-439f-a62d-4f45996b6d07",
    "id": "9f539354-ac81-4e3d-9082-b222d918d1bd",
    "isDefault": true,
    "managedByTenants": [],
    "name": "UdacityDS - 18",
    "state": "Enabled",
    "tenantId": "f958e84a-92b8-439f-a62d-4f45996b6d07",
    "user": {
      "name": "odl_user_230875@udacityhol.onmicrosoft.com",
      "type": "user"
    }
  }
]


### 2.2. Create Resource Group

In [5]:
! az group create \
    --name {RESOURCE_GROUP} \
    --location {LOCATION}

{
  "id": "/subscriptions/9f539354-ac81-4e3d-9082-b222d918d1bd/resourceGroups/azure-dev-udacity-prj2",
  "location": "southeastasia",
  "managedBy": null,
  "name": "azure-dev-udacity-prj2",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


### 2.3. Create Storage Account

In [6]:
! az storage account create \
    --name {STORAGE_ACCOUNT_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --location {LOCATION} \
    --sku {STORAGE_SKU}

{
  "accessTier": "Hot",
  "allowBlobPublicAccess": true,
  "allowCrossTenantReplication": null,
  "allowSharedKeyAccess": null,
  "allowedCopyScope": null,
  "azureFilesIdentityBasedAuthentication": null,
  "blobRestoreStatus": null,
  "creationTime": "2023-04-12T06:13:43.856050+00:00",
  "customDomain": null,
  "defaultToOAuthAuthentication": null,
  "dnsEndpointType": null,
  "enableHttpsTrafficOnly": true,
  "enableNfsV3": null,
  "encryption": {
    "encryptionIdentity": null,
    "keySource": "Microsoft.Storage",
    "keyVaultProperties": null,
    "requireInfrastructureEncryption": null,
    "services": {
      "blob": {
        "enabled": true,
        "keyType": "Account",
        "lastEnabledTime": "2023-04-12T06:13:44.215424+00:00"
      },
      "file": {
        "enabled": true,
        "keyType": "Account",
        "lastEnabledTime": "2023-04-12T06:13:44.215424+00:00"
      },
      "queue": null,
      "table": null
    }
  },
  "extendedLocation": null,
  "failoverInPro

### 2.4. Create Function App

In [7]:
! az functionapp create \
    --name {FUNCTION_NAME}  \
    --storage-account {STORAGE_ACCOUNT_NAME} \
    --consumption-plan-location {LOCATION} \
    --resource-group {RESOURCE_GROUP} \
    --functions-version {FUNCTION_VERSION} \
    --os-type {FUNCTION_OS_TYPE} \
    --runtime {FUNCTION_RUNTIME}

{
  "availabilityState": "Normal",
  "clientAffinityEnabled": false,
  "clientCertEnabled": false,
  "clientCertExclusionPaths": null,
  "clientCertMode": "Required",
  "cloningInfo": null,
  "containerSize": 0,
  "customDomainVerificationId": "2290CEE8851BEFF4B39F0DD5316902D8E8A2C6F7597FAD759CDA430C19BC121E",
  "dailyMemoryTimeQuota": 0,
  "defaultHostName": "neighborly-function.azurewebsites.net",
  "enabled": true,
  "enabledHostNames": [
    "neighborly-function.azurewebsites.net",
    "neighborly-function.scm.azurewebsites.net"
  ],
  "extendedLocation": null,
  "hostNameSslStates": [
    {
      "hostType": "Standard",
      "ipBasedSslResult": null,
      "ipBasedSslState": "NotConfigured",
      "name": "neighborly-function.azurewebsites.net",
      "sslState": "Disabled",
      "thumbprint": null,
      "toUpdate": null,
      "toUpdateIpBasedSsl": null,
      "virtualIp": null
    },
    {
      "hostType": "Repository",
      "ipBasedSslResult": null,
      "ipBasedSslState"

### 2.5. Create a CosmosDB manage MongoDB instance

#### 2.5.1. Create CosmosDB

In [8]:
! az cosmosdb create \
    -n {COSMOSDB_ACCOUNT_NAME} \
    -g {RESOURCE_GROUP} \
    --kind {COSMOSDB_KIND} \
    --server-version {COSMOSDB_SERVER_VERSION} \
    --default-consistency-level {COSMOSDB_DEFAULT_CONSISTENCY_LEVEL} \
    --enable-automatic-failover {COSMOSDB_ENABLE_AUTOMATIC_FAILOVER}

{
  "analyticalStorageConfiguration": {
    "schemaType": "FullFidelity"
  },
  "apiProperties": {
    "serverVersion": "4.2"
  },
  "backupPolicy": {
    "migrationState": null,
    "periodicModeProperties": {
      "backupIntervalInMinutes": 240,
      "backupRetentionIntervalInHours": 8,
      "backupStorageRedundancy": "Geo"
    },
    "type": "Periodic"
  },
  "capabilities": [
    {
      "name": "EnableMongo"
    }
  ],
  "capacity": null,
  "connectorOffer": null,
  "consistencyPolicy": {
    "defaultConsistencyLevel": "Eventual",
    "maxIntervalInSeconds": 5,
    "maxStalenessPrefix": 100
  },
  "cors": [],
  "createMode": null,
  "databaseAccountOfferType": "Standard",
  "defaultIdentity": "FirstPartyIdentity",
  "disableKeyBasedMetadataWriteAccess": false,
  "disableLocalAuth": false,
  "documentEndpoint": "https://neighborly-cosmosdb.documents.azure.com:443/",
  "enableAnalyticalStorage": false,
  "enableAutomaticFailover": false,
  "enableCassandraConnector": null,
  "ena

#### 2.5.2. Get CosmosDB Connection String

In [29]:
! az cosmosdb list-connection-strings --name {COSMOSDB_ACCOUNT_NAME} --resource-group {RESOURCE_GROUP}

{
  "connectionStrings": [
    {
      "connectionString": "mongodb://neighborly-cosmosdb:qEGPNRwDk63EF3cTPMRxXkVZ9J7pQ1oLjO4EkjfkpXhv29nJpVnomzPc6TA6TNvD18wnU3QxzPRUACDbsV32NQ==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@",
      "description": "Primary MongoDB Connection String"
    },
    {
      "connectionString": "mongodb://neighborly-cosmosdb:FDv9eHC9I4aHpww4Mc1coEdzssYYuWWRY7HfKvj9UjbKCkqEydpfH9iEGhBxODipTb6d8DC7MsqrACDb6W2Eug==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@",
      "description": "Secondary MongoDB Connection String"
    },
    {
      "connectionString": "mongodb://neighborly-cosmosdb:c4yt0VjbYETqqqNKMFFylHlWqcFQhBzV3vAPorANuERtgFWFlQhkyUYE5qjYlS1ZOV8dAaRtyisAACDbTm3eVA==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywri

In [28]:
! az cosmosdb keys list \
    --type connection-strings \
    --name {COSMOSDB_ACCOUNT_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --query 'connectionStrings[0].connectionString' \
    --output tsv

connectionStrings[0].connectionString


In [30]:
CONNECTION_STRING = "mongodb://neighborly-cosmosdb:qEGPNRwDk63EF3cTPMRxXkVZ9J7pQ1oLjO4EkjfkpXhv29nJpVnomzPc6TA6TNvD18wnU3QxzPRUACDbsV32NQ==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@"

#### 2.5.3. Create MongoDB Collections

1. Create advertisements collection

In [7]:
! printf '[{"key": {"keys": ["_id"]}}]' > idxpolicy-congdinh2023.json

'printf' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
%run variables.sh

In [21]:
! az cosmosdb mongodb collection create \
    -a neighborly-cosmosdb \
    -g azure-dev-udacity-prj2 \
    -d neighborlydb \
    -n advertisements \
    --shard _id \
    --throughput 400 \
    --idx @idxpolicy-congdinh2023.json

{
  "id": "/subscriptions/9f539354-ac81-4e3d-9082-b222d918d1bd/resourceGroups/azure-dev-udacity-prj2/providers/Microsoft.DocumentDB/databaseAccounts/neighborly-cosmosdb/mongodbDatabases/neighborlydb/collections/advertisements",
  "location": null,
  "name": "advertisements",
  "options": null,
  "resource": {
    "analyticalStorageTtl": null,
    "etag": null,
    "id": "advertisements",
    "indexes": [
      {
        "key": {
          "keys": [
            "_id"
          ]
        },
        "options": null
      }
    ],
    "rid": null,
    "shardKey": {
      "_id": "Hash"
    },
    "ts": null
  },
  "resourceGroup": "azure-dev-udacity-prj2",
  "tags": null,
  "type": "Microsoft.DocumentDB/databaseAccounts/mongodbDatabases/collections"
}


2. Create posts collection

In [22]:
! az cosmosdb mongodb collection create \
    -a neighborly-cosmosdb \
    -g azure-dev-udacity-prj2 \
    -d neighborlydb \
    -n posts \
    --shard _id \
    --throughput 400 \
    --idx @idxpolicy-congdinh2023.json

{
  "id": "/subscriptions/9f539354-ac81-4e3d-9082-b222d918d1bd/resourceGroups/azure-dev-udacity-prj2/providers/Microsoft.DocumentDB/databaseAccounts/neighborly-cosmosdb/mongodbDatabases/neighborlydb/collections/posts",
  "location": null,
  "name": "posts",
  "options": null,
  "resource": {
    "analyticalStorageTtl": null,
    "etag": null,
    "id": "posts",
    "indexes": [
      {
        "key": {
          "keys": [
            "_id"
          ]
        },
        "options": null
      }
    ],
    "rid": null,
    "shardKey": {
      "_id": "Hash"
    },
    "ts": null
  },
  "resourceGroup": "azure-dev-udacity-prj2",
  "tags": null,
  "type": "Microsoft.DocumentDB/databaseAccounts/mongodbDatabases/collections"
}


3. If you want to delete a collection, use:

In [17]:
! az cosmosdb mongodb collection delete \
    -a neighborly-cosmosdb \
    -d neighborlydb \
    -n posts \
    -g azure-dev-udacity-prj2 \
    --yes

In [18]:
! az cosmosdb mongodb collection delete \
    -a neighborly-cosmosdb \
    -d neighborlydb \
    -n advertisements \
    -g azure-dev-udacity-prj2 \
    --yes

4. View the collection

In [ ]:
! az cosmosdb mongodb collection list \
    --account-name neighborly-cosmosdb \
	--resource-group azure-dev-udacity-prj2 \
	--database-name neighborlydb

#### 2.5.4. Add data to collection

1. Add data to Advertisements Collection - Connection String from 2.5.2

In [31]:
print(CONNECTION_STRING)

mongodb://neighborly-cosmosdb:qEGPNRwDk63EF3cTPMRxXkVZ9J7pQ1oLjO4EkjfkpXhv29nJpVnomzPc6TA6TNvD18wnU3QxzPRUACDbsV32NQ==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@


In [6]:
! mongoimport \
    --uri f"{CONNECTION_STRING}" \
    --db {COSMOSDB_DATABASE_NAME} \
    --collection advertisements \
    --file='./sample_data/sampleAds.json' \
    --jsonArray

2023-04-12T13:39:53.244+0700	error parsing command line options: error parsing uri: scheme must be "mongodb" or "mongodb+srv"
2023-04-12T13:39:53.245+0700	try 'mongoimport --help' for more information


2. Add data to Posts Collection - Connection String from 2.5.2

In [33]:
! mongoimport \
    --uri f"{CONNECTION_STRING}" \
    --db {COSMOSDB_DATABASE_NAME} \
    --collection posts \
    --file='./sample_data/samplePosts.json' \
    --jsonArray

2023-04-12T14:33:21.238+0700	error parsing command line options: error parsing uri: scheme must be "mongodb" or "mongodb+srv"
2023-04-12T14:33:21.239+0700	try 'mongoimport --help' for more information


In [ ]:
from pymongo import MongoClient

In [ ]:
# Create a MongoDB client
client = MongoClient(f"{CONNECTION_STRING}")

# Access the database and collection
db = client[f"neighborlydb"]

In [ ]:
advertisements = db["advertisements"]

result = advertisements.find({})

# Print out the query result
for doc in result:
    print(doc)


In [ ]:
posts = db["posts"]

result = posts.find({})

# Print out the query result
for doc in result:
    print(doc)


### 2.6. Create App Service for Front-End

1. List Runtime Environment

In [ ]:
! az webapp list-runtimes

2. Create App Service Plan

In [ ]:
! az appservice plan create \
    --name {APP_SERVICE_PLAN_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --location {LOCATION} \
    --sku {APP_SKU} \
    --is-linux

3. Create App Service

In [ ]:
! az webapp create \
    --name {APP_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --plan {APP_SERVICE_PLAN_NAME} \
    --runtime {APP_RUNTIME}

## 3. Connect Function with CosmosDB

1. Verify the resources

In [ ]:
! az functionapp list  --resource-group {RESOURCE_GROUP}

2. Connect

In [ ]:
! az functionapp config appsettings set \
  --name neighborly-function \
  --resource-group azure-dev-udacity-prj2 \
  --setting MyDbConnection="mongodb://neighborly-cosmosdb:E2Uoovo8hP4chc62kH25aelu9oJkYraYdsvfav1V3lO74SiOg7mZeqATqwBkmfSkDkHN4NpcKKzfACDb6SfMCA==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@"

## 4. Deploy Functions

1. Test Functions at Local

In [ ]:
! func start

2. Publish functions

In [ ]:
! func azure functionapp publish {FUNCTION_NAME}

## 5 Dockerize Functions and Deploy to AKS

In [ ]:
! source variables.sh

### 5.1. Create a Dockerfile

In [ ]:
! func init --docker-only --python

### 5.2. Build the image using the Dockerfile

In [ ]:
! docker build -t $imageName:$imageTag .

In [ ]:
! docker run -p 7071:7071 -it $imageName:$imageTag

### 5.3. Create a repository in ACR service

1. Create Azure container registry

In [ ]:
! az acr create --resource-group $resourceGroup --name $containerRegistry --sku Basic

2. Login Azure container registry

In [ ]:
! az acr login --name $containerRegistry

3. Show container registry

In [ ]:
# Both these commands will give a same result
! az acr show --name $containerRegistry --query loginServer --output table
#! az acr list --resource-group $resourceGroup --query "[].{acrLoginServer:loginServer}" --output table

4. List docker images

In [ ]:
# List the local images
! docker images

5. Push image to Azure container registry

In [ ]:
# Tag the image with the same name as the ACR respository, else, the push will fail.
# SYNTAX 
# docker tag <name:tag> <ACR-respository>.azurecr.io/<name:tag>
! docker tag $imageName:$imageTag $containerRegistry.azurecr.io/$imageName:$imageTag

- Enable Admin User in Access Keys

In [ ]:
! az acr update --name $containerRegistry --admin-enabled true

- Verify Admin User Status

In [ ]:
! az acr show --name $containerRegistry --query "adminUserEnabled"

In [ ]:
# Go to Container Registry >> Settings >> Access Keys and enable the Admin user. 
# Use those credentials to login from your terminal. 
! docker login $containerRegistry.azurecr.io

In [ ]:
! docker push $containerRegistry.azurecr.io/$imageName:$imageTag

6. View Azure container registry image

In [ ]:
# View the newly pushed image in the ACR respository
! az acr repository list --name $containerRegistry --output table

### 5.3. Create an AKS cluster

1. Create an AKS cluster

In [ ]:
! az aks create \
    --name $AKSCluster \
    --resource-group $resourceGroup \
    --node-count 2 \
    --generate-ssh-keys \
    --attach-acr $containerRegistry \
    --location $region

2. Get credentials for your container service

In [ ]:
! az aks get-credentials \
    --name $AKSCluster \
    --resource-group $resourceGroup

3. Verify the connection to your cluster and view the cluster nodes using

In [ ]:
! kubectl get nodes
#Example output:
#NAME                                STATUS   ROLES   AGE     VERSION
#aks-nodepool1-38114521-vmss000000   Ready    agent   3m47s   v1.21.9
#aks-nodepool1-38114521-vmss000001   Ready    agent   3m45s   v1.21.9

4. Deploy the App to Kubernetes

In [ ]:
! func kubernetes deploy \
    --name $functionApp \
    --registry $containerRegistry \
    [--dry-run > deploy.yml]

In [ ]:
! kubectl config get-contexts

## 6. Delete Resources

In [ ]:
! az group delete -n {RESOURCE_GROUP} --verbose